In [2]:
import pandas as pd
import numpy as np
import pickle
root="data/"

In [3]:
train_sample = pickle.load(open(root+"nlfl_train_sample_v3.pickle", "rb"))
val_sample = pickle.load(open(root+"nlfl_val_sample_v3.pickle", "rb"))
test_sample = pickle.load(open(root+"nlfl_test_sample_v3.pickle", "rb"))

In [4]:
file_names = {
    "mf_train": root+"mf_features_train_task_C1.xlsx",
    "mf_test": root+"mf_features_test_task_C1.xlsx",
    "mf_val": root+"mf_features_val_task_C1.xlsx",
}

In [5]:
%%time
mf_A_train = pd.read_excel(file_names["mf_train"])
mf_A_val = pd.read_excel(file_names["mf_val"])
mf_A_test = pd.read_excel(file_names["mf_test"])

CPU times: user 2min 27s, sys: 281 ms, total: 2min 28s
Wall time: 2min 28s


In [6]:
%%time
mf_A_train = mf_A_train.set_index("id").loc[train_sample["id"]].reset_index()
mf_A_test = mf_A_test.set_index("id").loc[test_sample["id"]].reset_index()
mf_A_val = mf_A_val.set_index("id").loc[val_sample["id"]].reset_index()

CPU times: user 99.7 ms, sys: 52 ms, total: 152 ms
Wall time: 184 ms


In [7]:
cols_mf = [c for c in mf_A_train.columns if "linguistic" not in c and c != "id"]
cols_nlfl = [c for c in train_sample.columns if "chatgpt_" in c or c in ["id", "label_Q", "label"]]
cols = cols_mf + cols_nlfl

In [8]:
%%time
nlfl_hf_train = pd.concat([train_sample[cols_nlfl], mf_A_train[cols_mf]], axis=1)
nlfl_hf_test = pd.concat([test_sample[cols_nlfl], mf_A_test[cols_mf]], axis=1)
nlfl_hf_val = pd.concat([val_sample[cols_nlfl], mf_A_val[cols_mf]], axis=1)

CPU times: user 35.6 ms, sys: 20 ms, total: 55.6 ms
Wall time: 55 ms


In [9]:
from sklearn.tree import DecisionTreeClassifier

new_cols = [c for c in cols if c not in ["id", "label", "label_Q", "chatgpt_label"]] 
new_cols

['contextual<&>Q[rel_subj]+&A+',
 'contextual<&>Q[quién|cuál|qué]',
 'contextual<&>Q[ser&posible]',
 'contextual<&>A[binary(si|no)]',
 'contextual<&>Q[por&qué&equivocado]',
 'contextual<&>Q[ser*&bien]',
 'contextual<&>A[porque|binary(si|no)]',
 'contextual<&>Q[propn]+&A+',
 'contextual<&>Q[quién|cuál]',
 'contextual<&>Q[(ser*&correcto)|(tener&razón)]',
 'contextual<&>Q[ser*&correcto]',
 'contextual<&>Q[tener&razón]',
 'contextual<&>injection_index',
 'traditional<&>len(~A)',
 'traditional<&>prop_punct',
 'traditional<&>prop_punct+no-vowel',
 'traditional<&>prop_vowels',
 'traditional<&>len(tokens(A))',
 'traditional<&>len_max_rep_char',
 'semantic<&>A.isface()',
 'traditional<&>A.isdigit()',
 'traditional<&>frec_char(k)',
 'traditional<&>frec_char(g)',
 'traditional<&>frec_char(y)',
 'traditional<&>frec_char(j)',
 'traditional<&>frec_char(h)',
 'traditional<&>frec_char(x)',
 'traditional<&>frec_char(w)',
 'traditional<&>frec_char(ñ)',
 'semantic<&>A.is(nose)',
 'traditional<&>A.is(nan)

In [10]:
len(new_cols)

210

In [11]:
new_best_features = ['contextual<&>Q[quién|cuál|qué]',
 'contextual<&>A[binary(si|no)]',
 'contextual<&>Q[propn]+&A+',
 'contextual<&>Q[quién|cuál]',
 'contextual<&>Q[(ser*&correcto)|(tener&razón)]',
 'contextual<&>Q[ser*&correcto]',
 'contextual<&>Q[tener&razón]',
 'contextual<&>injection_index',
 'traditional<&>prop_punct',
 'semantic<&>A.isface()',
 'traditional<&>A.isdigit()',
 'traditional<&>frec_char(k)',
 'traditional<&>frec_char(y)',
 'traditional<&>frec_char(x)',
 'traditional<&>frec_char(ñ)',
 'semantic<&>A.is(nose)',
 'traditional<&>A.is(nan)',
 'semantic<&>A.is(ola|hola)',
 'semantic<&>A.contains(bad-word)',
 'semantic<&>A.contains(punct_faces)',
 'traditional<&>prop_no_math_punct',
 'traditional<&>max_vowel_rep_char_per_token',
 'traditional<&>num_tokens',
 'traditional<&>num_numbers',
 'traditional<&>num_math_punct',
 'semantic<&>num_keywords',
 'semantic<&>ratio_ud',
 'semantic<&>ratio_slang',
 'semantic<&>ratio_keywords',
 'traditional<&>ratio_vowel',
 'traditional<&>ratio_punct',
 'traditional<&>exist_numbs',
 'traditional<&>max_len_number']+ ['chatgpt_v7 (N)', 'chatgpt_v7 (Y)', 'chatgpt_v8 (N)',
       'chatgpt_v9 (N)', 'chatgpt_v9 (Y)', 'chatgpt_v10 (Y)',
       'chatgpt_v5_2 (Y)', 'chatgpt_v5_4 (Y)', 'chatgpt_v6_1 (Y)',
       'chatgpt_Q3_4 (Y)', 'chatgpt_Q3_5 (Y)', 'chatgpt_Q3_7 (Y)',
       'chatgpt_Q3_7_e1 (Y)', 'chatgpt_10_e1 (N)', 'chatgpt_10_e2 (N)',
       'chatgpt_10_e2 (Y)', 'chatgpt_10_e3 (N)', 'chatgpt_10_e3 (Y)',
       'chatgpt_10_e5 (N)', 'chatgpt_7_e1 (Y)', 'chatgpt_7_e2 (N)',
       'chatgpt_7_e2 (Y)', 'chatgpt_7_e3 (N)', 'chatgpt_7_e5 (N)',
       'chatgpt_Q3_4_e2 (N)', 'chatgpt_Q3_5_e2 (N)',
       'chatgpt_Q3_5_e4 (Y)', 'chatgpt_Q3_5_e5 (Y)', 'chatgpt_HF_1 (Y)',
       'chatgpt_HF_2 (N)', 'chatgpt_HF_2 (Y)', 'chatgpt_HF_3 (Y)',
       'chatgpt_HF_4 (Y)', 'chatgpt_HF_6 (Y)', 'chatgpt_SP_1 (N)',
       'chatgpt_SP_3 (N)', 'chatgpt_SP_3 (Y)', 'chatgpt_SP_5 (Y)',
       'chatgpt_SP_6 (Y)']

In [12]:
clf = DecisionTreeClassifier(random_state=42, min_impurity_decrease=0.0000, max_depth=5, criterion="gini")
clf.fit(nlfl_hf_train[new_best_features], nlfl_hf_train["label"])

DecisionTreeClassifier(max_depth=5, random_state=42)

In [13]:
from matplotlib import pyplot as plt
from sklearn import tree

In [14]:
[new_best_features[i] for i in clf.tree_.feature if i>=0]

['traditional<&>num_tokens',
 'contextual<&>Q[propn]+&A+',
 'chatgpt_SP_6 (Y)',
 'traditional<&>max_vowel_rep_char_per_token',
 'chatgpt_SP_3 (Y)',
 'traditional<&>exist_numbs',
 'contextual<&>A[binary(si|no)]',
 'traditional<&>exist_numbs',
 'contextual<&>Q[ser*&correcto]',
 'chatgpt_7_e3 (N)',
 'chatgpt_HF_2 (N)',
 'chatgpt_Q3_5_e5 (Y)',
 'chatgpt_7_e3 (N)',
 'contextual<&>Q[ser*&correcto]',
 'chatgpt_v8 (N)',
 'chatgpt_v8 (N)',
 'chatgpt_10_e2 (N)',
 'traditional<&>ratio_vowel',
 'contextual<&>A[binary(si|no)]',
 'chatgpt_HF_2 (Y)',
 'semantic<&>ratio_ud',
 'chatgpt_v8 (N)',
 'chatgpt_10_e5 (N)',
 'traditional<&>ratio_vowel',
 'traditional<&>ratio_punct',
 'chatgpt_HF_1 (Y)',
 'chatgpt_Q3_4 (Y)',
 'traditional<&>num_tokens',
 'contextual<&>Q[(ser*&correcto)|(tener&razón)]',
 'traditional<&>max_vowel_rep_char_per_token']

In [15]:
fig, ax = plt.subplots(1, 1, figsize=(265, 85))
tree.plot_tree(clf, feature_names=new_best_features, ax=ax, fontsize=45, class_names=["C0","C1"]);

In [1]:
path = [
    ('traditional<&>num_tokens', 3.5, False),
    ('chatgpt_10_e2 (N)', 0.147, True),
    # ('traditional<&>ratio_vowel', 0.326, True),
    # ('traditional<&>ratio_punct', 0.146, True),
    # ('chatgpt_Q3_4 (Y)', 0.437, False),
    # ('chatgpt_HF_1 (Y)', 0.118, False),
    ('traditional<&>ratio_vowel', 0.414, False),
    ('semantic<&>ratio_ud', 0.141, True),
    ('chatgpt_v8 (N)', 0.439, True)
    # ('chatgpt_10_e5 (N)', 0.158, False)
]
for ix in nlfl_hf_train.index:
    Q, A, label = train_sample.loc[ix]["Q A label".split()]
    row = nlfl_hf_train.loc[ix]
    if all([(row[p[0]] <= p[1]) == p[2] for p in path]):
        print([row[p[0]] for p in path])
        print(ix, Q, str(A).replace("\n", " "), label, clf.predict([nlfl_hf_train.loc[ix][new_best_features]]), sep="\n")

NameError: name 'nlfl_hf_train' is not defined

In [17]:
from sklearn.metrics import classification_report

In [18]:
print(classification_report(nlfl_hf_train["label"], clf.predict(nlfl_hf_train[new_best_features]), digits=4))

              precision    recall  f1-score   support

         0.0     0.9509    0.9777    0.9641     10019
         1.0     0.8226    0.6714    0.7394      1540

    accuracy                         0.9369     11559
   macro avg     0.8867    0.8246    0.8517     11559
weighted avg     0.9338    0.9369    0.9342     11559



In [19]:
val_sub_index = nlfl_hf_val.index
print(classification_report(nlfl_hf_val.loc[val_sub_index]["label"], clf.predict(nlfl_hf_val.loc[val_sub_index][new_best_features]), digits=4))

              precision    recall  f1-score   support

           0     0.9443    0.9725    0.9582      2510
           1     0.7796    0.6289    0.6961       388

    accuracy                         0.9265      2898
   macro avg     0.8619    0.8007    0.8272      2898
weighted avg     0.9222    0.9265    0.9231      2898



In [20]:
sub_index_test = nlfl_hf_test[nlfl_hf_test["label_Q"] == 3].index
print(classification_report(nlfl_hf_test.loc[sub_index_test]["label"], clf.predict(nlfl_hf_test.loc[sub_index_test][new_best_features]), digits=4))

              precision    recall  f1-score   support

           0     0.9701    0.9924    0.9811       262
           1     0.9429    0.8049    0.8684        41

    accuracy                         0.9670       303
   macro avg     0.9565    0.8986    0.9248       303
weighted avg     0.9665    0.9670    0.9659       303

